In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import sklearn.metrics

df = pd.read_csv('minitree_4b_2_26.txt', sep = ' ', header=0)

y = (df['Jet_genjetPt']/df['Jet_pt']).values.reshape(-1, 1)
X = df.drop(['Jet_pt', 'Jet_genjetPt'], axis = 1).values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
X_train = X[0:3200000]
y_train = y[0:3200000]
X_test = X[3200000:]
y_test = y[3200000:]

learning_rate = 0.000001
batch_size = 200
epochs = 10
g_1 = tf.Graph()
n, d = X_train.shape

with g_1.as_default():
    
    X_placeholder = tf.placeholder(tf.float32, [None, 17])
    y_placeholder = tf.placeholder(tf.float32, [None, 1])

    a1 = tf.layers.dense(X_placeholder, 40, tf.nn.relu, name = 'layer_1')
    a2 = tf.layers.dense(a1, 40, tf.nn.relu, name = 'layer_2')
    a3 = tf.layers.dense(a2, 40, tf.nn.relu, name = 'layer_3')
    a4 = tf.layers.dense(a3, 40, tf.nn.relu, name = 'layer_4')
    a5 = tf.layers.dense(a4, 40, tf.nn.relu, name = 'layer_5')
    a6 = tf.layers.dense(a5, 40, tf.nn.relu, name = 'layer_6')
    a7 = tf.layers.dense(a6, 40, tf.nn.relu, name = 'layer_7')
    a8 = tf.layers.dense(a7, 20, tf.nn.relu, name = 'layer_8')
    a9 = tf.layers.dense(a8, 10, name = 'layer_9', activation = None)
    z10 = tf.layers.dense(a9, 1, name = 'layer_10', activation = None)

    loss = tf.losses.mean_squared_error(labels = y_placeholder, predictions = z10)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    #initializer
    init = tf.global_variables_initializer()

    config = tf.ConfigProto(allow_soft_placement = False, log_device_placement=True)
    #config.gpu_options.per_process_gpu_memory_fraction = 0.4
    config.gpu_options.allow_growth = True
    
    #saver
    saver = tf.train.Saver()
    with tf.Session(config = config) as sess:
        sess.run(init)
        #saver.restore(sess, "/saver/model.ckpt")
        for epoch in range(epochs):
            for batch in range(int (n / batch_size)):
                batch_xs = X_train[(batch*batch_size) : (batch+1)*batch_size]
                batch_ys = y_train[(batch*batch_size) : (batch+1)*batch_size]
                sess.run(train_step, feed_dict = {X_placeholder:batch_xs, y_placeholder:batch_ys})
                
                sys.stdout.write("\rBatch(10):" + str(sess.run(loss, feed_dict = {X_placeholder:batch_xs, y_placeholder:batch_ys})))

            print("")
        saver.save(sess, "/saver/model.ckpt")
        pred = sess.run(z10, feed_dict = {X_placeholder:X_test})
        print('\nR2_score: ', sklearn.metrics.r2_score(y_true = y_test, y_pred = pred))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Batch(10):0.137339286